In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import librosa
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,LSTM,GlobalMaxPool1D, Bidirectional 
from tensorflow.keras.models import Sequential
from tensorflow.keras.metrics import Precision
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
from numpy import random
from random import choices
import os
from sklearn.utils import shuffle
from sklearn import metrics
import pickle
from scipy.io.wavfile import read,write
import pickle

from datetime import datetime
from packaging import version

tf.keras.backend.set_floatx('float32')

print("TensorFlow version: ", tf.__version__)

TensorFlow version:  2.4.1


In [ ]:
os.chdir('/content/drive/My Drive/')
os.listdir()

['depthwise_conv2d.py',
 'Colab Notebooks',
 'logs',
 'Pickles',
 'Zips',
 'Data',
 'Help_Models',
 'Bachao_Models',
 'model.py',
 'data_preprocess.py',
 'utils.py',
 'CRoam Model Training  Report.pdf',
 'speaker_ver1_model.h5',
 'speaker_ver1_model.pb',
 'speaker_ver1a_model.h5',
 'speaker_ver1b_model.h5',
 'speaker_ver1a_model.pb',
 'graph_def_for_reference.pb.ascii',
 'speaker_ver1b_model.pb',
 'new_mix_help_train_x.pickle']

# To train on new data and dumping into a pickle file

In [ ]:
os.chdir('Data/Bachao')
os.listdir()


['Bachao_Data_Babble_10dB_Trimmed',
 'Bachao_Data_Natural_10dB_Trimmed',
 'Bachao_Data_Old_Trimmed',
 'Bachao_Data_Test_Trimmed',
 'Bachao_Data_Choir',
 'Bachao_Data_Arijit',
 'Bachao_Data_Impala',
 'Bachao_Data_Party',
 'Bachao_Data_Cheerful',
 'Bachao_Data_Chatter',
 'Bachao_Data_Market',
 'Bachao_Data_Lecture',
 'Bachao_Data_Dishes',
 'Bachao_Data_Bike',
 'Bachao_Data_Pink',
 'Bachao_Data_Tap']

In [ ]:
os.chdir('../../')

In [ ]:
# KEYWORD_FOLDER1 = 'Data/Bachao/Bachao_Data_Old_Trimmed'
# KEYWORD_FOLDER2 = 'Data/Bachao/Bachao_Data_Natural_10dB_Trimmed'
# KEYWORD_FOLDER3 = 'Data/Bachao/Bachao_Data_Babble_10dB_Trimmed'
# KEYWORD_FOLDER4 = 'Data/Bachao/Bachao_Data_Test_Trimmed'

KEYWORD_FOLDER1 = 'Data/Bachao/Bachao_Data_Arijit'
KEYWORD_FOLDER2 = 'Data/Bachao/Bachao_Data_Party'
KEYWORD_FOLDER3= 'Data/Bachao/Bachao_Data_Market'
KEYWORD_FOLDER4 = 'Data/Bachao/Bachao_Data_Lecture'
KEYWORD_FOLDER5 = 'Data/Bachao/Bachao_Data_Impala'
KEYWORD_FOLDER6= 'Data/Bachao/Bachao_Data_Choir'
KEYWORD_FOLDER7 = 'Data/Bachao/Bachao_Data_Cheerful'
KEYWORD_FOLDER8= 'Data/Bachao/Bachao_Data_Chatter'
KEYWORD_FOLDER9= 'Data/Bachao/Bachao_Data_Dishes'
KEYWORD_FOLDER10= 'Data/Bachao/Bachao_Data_Pink'
KEYWORD_FOLDER11= 'Data/Bachao/Bachao_Data_Bike'
KEYWORD_FOLDER12= 'Data/Bachao/Bachao_Data_Tap'

# KEYWORD_FOLDER4= 'Data/Help/Google_noise_mix_pink'
# KEYWORD_FOLDER5= 'Data/Help/Google_noise_mix_tap'
# KEYWORD_FOLDER6 = 'Data/Help/Google_noise_mix_dishes'
# KEYWORD_FOLDER7 = 'Data/Help/Google_noise_mix_bike'


NEGATIVE_FOLDER1 = 'Data/Negative/Negative_Data/'
NEGATIVE_FOLDER2 = 'Data/Negative/Negative_Data_10dB'
NEGATIVE_FOLDER3 = 'Data/Negative/Negative_Data_Natural_10dB'

# KEYWORD_FOLDER_TEST = 'Data/Help/Crowd-Sourced'
# KEYWORD_FOLDER_TEST = 'Data/Help/Help_Data_Test_Old/'
# NEGATIVE_FOLDER_TEST = 'Data/Negative/Negative_Data_Test_Old/'

INPUT_SHAPE = (126,40)

In [ ]:
def count_files(folder, extension):
	count = 0
	for file in os.listdir(folder):
		if file.endswith(extension):
			file_path = os.path.join(folder, file)
			count += 1
	return count

In [ ]:
def load_data_folder(folder, is_keyword):
  num_samples = count_files(folder, '.wav')
  data_X = np.zeros((num_samples, INPUT_SHAPE[0], INPUT_SHAPE[1]), dtype=np.float64)
  data_Y = np.zeros((num_samples), dtype=np.float64)

  count = 0
  for file in os.listdir(folder):
    if file.endswith('.wav'):
      file_path = os.path.join(folder, file)
      y, sr = librosa.load(file_path,sr=None)
      mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=128, n_fft=256, n_mfcc=20)
      mfcc_delta = librosa.feature.delta(mfcc)[:10, :]
      mfcc_double_delta = librosa.feature.delta(mfcc, order=2)[:10, :]
      data_X[count, :, :20] = mfcc.T
      data_X[count, :, 20:30] = mfcc_delta.T
      data_X[count, :, 30:] = mfcc_double_delta.T
      data_Y[count] = int(is_keyword)
      count += 1
      if count%50==0:
        print(count)
  return data_X, data_Y

In [ ]:
def load_data(folders):
	num_samples = sum([count_files(folder, '.wav') for folder, is_keyword in folders])
	data_X = np.zeros((num_samples, INPUT_SHAPE[0], INPUT_SHAPE[1]), dtype=np.float64)
	data_Y = np.zeros((num_samples), dtype=np.float64)
	count = 0
	try:
		for folder, is_keyword in folders:
			num_samples_folder = count_files(folder, '.wav')
			data_X[count:count+num_samples_folder, :, :], data_Y[count:count+num_samples_folder] = (
				load_data_folder(folder, is_keyword))
			count += num_samples_folder
		return shuffle(data_X, data_Y, random_state=0)
	except:
		print('error')


In [ ]:
def load_train_data():
  #folders = [(NEGATIVE_FOLDER_TRAIN_1, False), (NEGATIVE_FOLDER_TRAIN_2, False), (NEGATIVE_FOLDER_TRAIN_3, False)]
  folders = [(KEYWORD_FOLDER1, True), (KEYWORD_FOLDER2, True), (KEYWORD_FOLDER3, True), (KEYWORD_FOLDER4, True), (KEYWORD_FOLDER5, True), (KEYWORD_FOLDER6, True), (KEYWORD_FOLDER7, True), (KEYWORD_FOLDER8, True), (KEYWORD_FOLDER9, True), (KEYWORD_FOLDER10, True), (KEYWORD_FOLDER11, True), (KEYWORD_FOLDER12, True)]
  # folders = [(KEYWORD_FOLDER1, True), (KEYWORD_FOLDER2, True), (KEYWORD_FOLDER3, True), (KEYWORD_FOLDER4, True), (NEGATIVE_FOLDER1, False), (NEGATIVE_FOLDER2, False), (NEGATIVE_FOLDER3, False)]
  return load_data(folders)

In [ ]:
def load_test_data():
  #folders = [(NEGATIVE_FOLDER_TEST, True)]
  folders = [(KEYWORD_FOLDER_TEST, True)] 
  return load_data(folders)

In [ ]:
#Generate the MFCC matrix
train_X0, train_Y0 = load_train_data()
print("Train data extracted")

# test_X, test_Y = load_test_data()
# print("Test data extracted")

/usr/local/lib/python3.6/dist-packages/librosa/filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "


50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
Train data extracted


In [ ]:
# Dump into a pickle
import pickle

with open("Pickles/noise_bachao_train_x.pickle", "wb") as f:
  pickle.dump(train_X0, f)
print("Train set features done")
with open("Pickles/noise_bachao_train_y.pickle", "wb") as f:
  pickle.dump(train_Y0, f)
print("Train set labels done")


# with open("help_trimmed_test_x.pickle", "wb") as f:
#   pickle.dump(test_X, f)
# print("Test set features done")
# with open("help_trimmed_test_y.pickle", "wb") as f:
#   pickle.dump(test_Y, f)
# print("Test set labels done")

FileNotFoundError: ignored

#To load data from an existing pickle

In [ ]:
os.chdir("Pickles")
os.listdir()

NameError: ignored

In [ ]:
# Load from a pickle

# with open('crowdsourced_train_x.pickle', 'rb') as f:
#   train_X0cs = pickle.load(f)
# print("Crowdsourced Train set features loaded")
# with open('crowdsourced_train_y.pickle', 'rb') as f:
#   train_Y0cs = pickle.load(f)
# print("Crowdsourced Train set labels loaded")

with open('Bachao/noise_bachao_train_x.pickle', 'rb') as f:
  train_X0b1 = pickle.load(f)
print("Noisy Bachao Train set features loaded")
with open('Bachao/noise_bachao_train_y.pickle', 'rb') as f:
  train_Y0b1 = pickle.load(f)
print("Noisy Bachao Train set labels loaded")

with open('Negative/youtube_train_x(1).pickle', 'rb') as f:
  train_X0y1 = pickle.load(f)
print("YouTube(1) Train set features loaded")
with open('Negative/youtube_train_y(1).pickle', 'rb') as f:
  train_Y0y1 = pickle.load(f)
print("YouTube(1) Train set labels loaded")

with open('Negative/google_trimmed_train_x.pickle', 'rb') as f:
  train_X0g = pickle.load(f)
print("Google Train set features loaded")
with open('Negative/google_trimmed_train_y.pickle', 'rb') as f:
  train_Y0g = pickle.load(f)
print("Google Train set labels loaded")

with open('Bachao/old_bachao_train_x.pickle', 'rb') as f:
  train_X0b = pickle.load(f)
print("Train set features loaded")
with open('Bachao/old_bachao_train_y.pickle', 'rb') as f:
  train_Y0b = pickle.load(f)
print("Train set labels loaded")

FileNotFoundError: ignored

In [ ]:
# print(train_X0cs.shape, train_Y0cs.shape) #all true
print(train_X0b1.shape, train_Y0b1.shape) #all true
print(train_X0y1.shape, train_Y0y1.shape) #all false
print(train_X0g.shape, train_Y0g.shape)   #all false
print(train_X0b.shape, train_Y0b.shape)   #2588/3784 trues

In [ ]:
# print(train_Y0cs.sum())
print(train_Y0b1.sum())
print(train_Y0y1.sum())
print(train_Y0g.sum())
print(train_Y0b.sum())

In [ ]:
print(train_X0b1.shape)
for i in train_X0b1[20][50]:
  print(i)

In [ ]:
train_X0 = np.concatenate((train_X0y1[:15000], train_X0g[:10000], train_X0b))
train_Y0 = np.concatenate((train_Y0y1[:15000], train_Y0g[:10000], train_Y0b))

test_X0 = np.concatenate((train_X0b1, train_X0y1[15000:], train_X0g[10000:]))
test_Y0 = np.concatenate((train_Y0b1, train_Y0y1[15000:], train_Y0g[10000:]))

print(train_X0.shape, train_Y0.shape)
print(train_Y0.sum())
print(test_X0.shape, test_Y0.shape)
print(test_Y0.sum())

(28784, 126, 40) (28784,)
2588.0
(29491, 126, 40) (29491,)
10257.0


In [ ]:
randomize = np.arange(len(train_Y0))
np.random.shuffle(randomize)
train_X_shuffle = train_X0[randomize]
train_Y_shuffle = train_Y0[randomize]

In [ ]:
import pickle

with open("total_bachao_x_new.pickle", "wb") as f:
  pickle.dump(train_X_shuffle, f)
print("Train set features done")
with open("total_bachao_y_new.pickle", "wb") as f:
  pickle.dump(train_Y_shuffle, f)
print("Train set labels done")

Train set features done
Train set labels done


# Training Model




In [ ]:
os.chdir("Pickles")
os.listdir()

['Help', 'Bachao', 'Negative']

In [ ]:
with open('Bachao/total_bachao_x_new.pickle', 'rb') as f:
  train_X0 = pickle.load(f)
print("Train set features loaded")
with open('Bachao/total_bachao_y_new.pickle', 'rb') as f:
  train_Y0 = pickle.load(f)
print("Train set labels loaded")

# with open('crowdsourced_train_x.pickle', 'rb') as f:
#   train_X0cs = pickle.load(f)
# print("Crowdsourced Train set features loaded")
# with open('crowdsourced_train_y.pickle', 'rb') as f:
#   train_Y0cs = pickle.load(f)
# print("Crowdsourced Train set labels loaded")

Train set features loaded
Train set labels loaded


In [ ]:
print(train_X0.shape, train_Y0.shape)
print(train_Y0.sum())

(58275, 126, 40) (58275,)
12845.0


In [ ]:
# randomize = np.arange(len(train_Y0))
# np.random.shuffle(randomize)
# train_X_shuffle = train_X0[randomize]
# train_Y_shuffle = train_Y0[randomize]

In [ ]:
print(train_X_shuffle.shape)
print(train_Y_shuffle.shape)

(28784, 126, 40)
(28784,)


In [ ]:
import sklearn
train_X_cv, test_X_cv, train_Y_cv, test_Y_cv = sklearn.model_selection.train_test_split(train_X_shuffle, train_Y_shuffle, test_size=0.1)
# train_X_cv, test_X_cv, train_Y_cv, test_Y_cv = sklearn.model_selection.train_test_split(train_X0, train_Y0, test_size=0.1)

In [ ]:
test_Y_cv.shape

(2879,)

In [ ]:
class_weight = {True: 4.,
                False: 1.}

In [ ]:
logdir = "../logs/Bachao/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

inputs = keras.Input(shape = (126,40))    
x = Bidirectional(LSTM(units=128,return_sequences=True))(inputs)
x = GlobalMaxPool1D()(x)
# x = Dense(64, activation='sigmoid')(x)
outputs = Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs, outputs)
model.summary()
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['acc', Precision()])

training_history = model.fit(train_X_cv, train_Y_cv, class_weight = class_weight, epochs=10, validation_split=0.2, callbacks=[tensorboard_callback])
print("Average test loss: ", np.average(training_history.history['loss']))

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 126, 40)]         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 126, 256)          173056    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 173,313
Trainable params: 173,313
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
648/648 [==============================] - 151s 228ms/step - loss: 0.3310 - acc: 0.9521 - precision_1: 0.7515 - val_loss: 0.0500 - val_acc: 0.9869 - val_precision_1: 0.8793
Epoch 2/10
648/648 [==============================] - 148s 228ms/st

In [ ]:
#Uncomment to save the retrained model
model.save('../Bachao_Models/HDF5/bachao4.h5')
print("SAVED")

SAVED


# If Data Generator required

In [ ]:
def generator(features, labels, batch_size):
    # Create empty arrays to contain batch of features and labels#
    batch_features = np.zeros((batch_size, 126, 40))
    batch_labels = np.zeros((batch_size,1))
    while True:
        for i in range(batch_size):
            index= random.choice(len(features),1)
            batch_features[i] = features[index]
            batch_labels[i] = labels[index]
        yield batch_features, batch_labels

In [ ]:
inputs = keras.Input(shape = (126,40))    
x = Bidirectional(LSTM(units=128,return_sequences=True))(inputs)
x = GlobalMaxPool1D()(x)
outputs = Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs, outputs)
model.summary()
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['acc', Precision()])

model.fit_generator(generator(train_X0[10000:], train_Y0[10000:], 128), verbose=1, steps_per_epoch=320, epochs=50, validation_data=(train_X0[:10000], train_Y0[:10000]))

In [ ]:
# def create_model(INPUT_SHAPE):
#   print ('Creating model...')
#   model = Sequential()
#   model.add(Bidirectional(LSTM(units=128, input_shape=INPUT_SHAPE, return_sequences=True)))
#   model.add(GlobalMaxPool1D())
#   model.add(Dense(units=1,activation='sigmoid'))
#   opt = keras.optimizers.Adam(learning_rate=0.001)	
#   model.compile(loss='binary_crossentropy',
# 	              optimizer=opt,
# 	              metrics=['acc', Precision()])
#   return model

# model = create_model(INPUT_SHAPE = (126,40))

# training_history = model.fit(train_X_cv, train_Y_cv, epochs=5, validation_split=0.2, callbacks=[tensorboard_callback])
# print("Average test loss: ", np.average(training_history.history['acc']))

In [ ]:
%reload_ext tensorboard
%load_ext tensorboard
%tensorboard --logdir /content/drive/'My Drive'/logs/Bachao

# If Loading a Model

In [ ]:
modelPath = '../Bachao_Models/HDF5/bachao1.h5'
model = tf.keras.models.load_model(modelPath)
model.summary()

# Evaluation

In [ ]:
predictions = model.predict(test_X0)
y_pred = (predictions>0.95)
matrix = metrics.confusion_matrix(test_Y0, y_pred)
print(matrix)


In [ ]:
c=0
for i in range(y_pred.shape[0]):
  if y_pred[i]==True and test_Y_cv[i]==False:
    c=c+1
print(c)

2
